# Bi-Directional Attention Flow Model

#### Set Checkpoint and Data Folder

In [1]:
import sys, os
import torch
pwd = os.getcwd()

class Arguments():
    data = os.path.join(pwd, 'DATA', 'train_v2.1.json')
    exp_folder = os.path.join(pwd, 'neuer_Test')
    word_rep = os.path.join(pwd, 'DATA', 'glove.840B.300d.txt')
    cuda = torch.cuda.is_available()
    use_covariance = False
    force_restart = False

args = Arguments()

if not os.path.exists(args.exp_folder):
    os.makedirs(args.exp_folder)


#### Global Configurations (instead of config.yaml)

In [2]:
import yaml

config_yaml = """
    bidaf:
        dropout: 0.2
        num_highways: 2
        num_lstm: 2
        hidden_size: 100
        embedding_dim: 300
        embedding_reduce: 100
        characters:
            dim: 16
            num_filters: 100
            filter_sizes:
                - 5
    training:
        lr: 0.001
        betas:
            - 0.9
            - 0.999
        eps: 0.00000001
        weigth_decay: 0
        epochs: 1
        batch_size: 60
        limit: 400
"""
config = yaml.load(config_yaml, Loader=yaml.FullLoader)


#### Importing the MSMARCO Bidaf Model

In [3]:
sys.path.append(os.path.join(pwd,'MsmarcoQuestionAnswering','Baseline'))
sys.path.append(os.path.join(pwd,'MsmarcoQuestionAnswering','Baseline','scripts'))

import MsmarcoQuestionAnswering.Baseline.mrcqa as mrcqa
import MsmarcoQuestionAnswering.Baseline.scripts.dataset as dataset
import MsmarcoQuestionAnswering.Baseline.scripts.checkpointing as checkpointing
import MsmarcoQuestionAnswering.Baseline.scripts.train as manager

#### Initialize Model, Load and Tokenize Data, Getting ID-->Token and ID-->Char Mapping

In [4]:
import h5py

checkpoint, training_state, epoch = manager.try_to_resume(args.force_restart, args.exp_folder)
if checkpoint:
        print('Resuming training...')
        model, id_to_token, id_to_char, optimizer, data = manager.reload_state(checkpoint, training_state, config, args, None)
else:
        print('Preparing to train...')
        model, id_to_token, id_to_char, optimizer, data = manager.init_state(config, args,None)
        checkpoint = h5py.File(os.path.join(args.exp_folder, 'checkpoint'))
        checkpointing.save_vocab(checkpoint, 'vocab', id_to_token)
        checkpointing.save_vocab(checkpoint, 'c_vocab', id_to_char)
        checkpointing.checkpoint(model,0,optimizer,checkpoint, args.exp_folder)

Preparing to train...
Load Data [1/6]
Start Organizing Data...
Organizing progress: 0.0 x 10⁴/80 x 10⁴
Organizing progress: 10.0 x 10⁴/80 x 10⁴
Organizing progress: 20.0 x 10⁴/80 x 10⁴
Organizing progress: 30.0 x 10⁴/80 x 10⁴
Organizing progress: 40.0 x 10⁴/80 x 10⁴
Organizing progress: 50.0 x 10⁴/80 x 10⁴
Organizing progress: 60.0 x 10⁴/80 x 10⁴
Organizing progress: 70.0 x 10⁴/80 x 10⁴
Organizing progress: 80.0 x 10⁴/80 x 10⁴
Tokenize Data [2/6]
0.0 x 10⁴/35.024 x 10⁴
1.0 x 10⁴/35.024 x 10⁴
2.0 x 10⁴/35.024 x 10⁴
3.0 x 10⁴/35.024 x 10⁴
4.0 x 10⁴/35.024 x 10⁴
5.0 x 10⁴/35.024 x 10⁴


KeyboardInterrupt: 

In [ ]:
if args.cuda:
    data.tensor_type = torch.cuda.LongTensor